### <center>Amplicación de análisis de estructuras. Máster Universitario en Ingeniería de Caminos, Canales y Puertos. Universidad de Granada. Curso 2023-2024<center>

# <center>Software para la resolución numérica del Problema de Placas Rectangulares mediante el Método de Levy.<center>

#### <center>Antonio Gomez. Marzo 2024<center>

Este script de Jupyter lab contiene una serie de algoritmos escritos en lenguaje Python para calcular y evaluar la flecha w(x,y) mediante el método de Levy para poder resolver la tarea adjunta.

Los casos disponibles de resolución son los de:
- Pacla tetrapoyada con carga uniforme de valor p.
<img src="carga_uniforme.png" alt="Texto alternativo" width="400">

- Placa tetrapoyada con carga de ley lineal antimétrica de valor máximo q.
<img src="carga_lineal_x.png" alt="Texto alternativo" width="400">

Tendrás que superponer flechas debidas a diferentes estados de carga para resolver las preguntas de la tarea. Superponer flechas significa sumar matrices de flechas w_xy debidas a los diferentes estados de carga involucrados en tu caso.

**Índice:**
1. Importamos las librerías necesarias
2. Datos de entrada
3. Flecha generada en placa tetrapoyada por una carga uniforme de valor p w<sup>S</sup><sub>p</sub>(x,y)
4. Flecha generada en placa tetrapoyada por una carga lineal de valor máximo q w<sup>A</sup><sub>q</sub>(x,y)


<br>

**1. Importamos las librerías necesarias**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

<br>

**2. Datos de entrada** (Introducir unidades en el sistema internacional)

In [ ]:
# Datos de la placa:
a = ...
b = ...
h = ...
rho_placa = ...
nu = ...
E = ...
I = h**3/12
D = E*I/(1-nu**2)

# Datos del sensor: coordenadas x,y del sensor
x_sensor = ...
y_sensor = ...

# Carga uniforme:
p = ... #Cuidado, si la carga es hacia abajo hay que indicar signo negativo

# Carga lineal, valor máximo:
q = ... #Cuidado, si la carga es hacia abajo hay que indicar signo negativo

# Valores de discretización espacial
dx = 2e-2
dy = 2e-2

# Recuerda que en Python, el primer elemento de un vector tiene indice 0. Igual en una matriz, donde su primer elemento es el (0,0)
# Recuerda que puedes usar la función print() para ver el valor de alguna variable.
# Puedes también usar la función np.shape() para ver el tamaño de matrices, o alternativamente la funcion np.size("matrix",0), 0 para la dimensión filas y 1 para la dimensión 1.
# La funcion len() para saber la longitud de un vector.

Discretización del dominio espacial de la placa

In [ ]:
# Creamos dos vectores x e y, que contienen los valores discretos del espacio donde se evaluaran los cálculos
x = np.arange(-a/2, a/2+dx, dx) 
y = np.arange(0, b+dy, dy)

Declaración de terminos de los sumatorios

In [ ]:
# Creamos un vector de enteros n
nmax = 3
nmaxreal = nmax if nmax % 2 != 0 else nmax + 1 # sí nmax es par, haz que el nmaxreal sea el siguiente impar
n = np.arange(1,nmaxreal+1,2) #vector de 1 a nmax con incrementos de 2, es decir, solo impares

# Definimos el número de elementos n y m a evaluar
neval = nmaxreal

# Esto es un mensaje de error en caso de indicar neval y meval mayores que los máximos preestablecidos.
if neval > nmaxreal:
    raise ValueError('El número máximo de n a evaluar introducido es mayor que el n máximo preestablecido. Introduce un número n a evaluar menor.')

# Creamos dos matrices N y X, de forma que seleccionando un mismo elemento en ambas tenemos una dupla n,x
N, X = np.meshgrid(n, x)

# Calculamos las constantes de Levy $\lambda_n$ y $\alpha_n$
lambda_n = N*np.pi/b
alpha_n = lambda_n*a/2

<br>

**3. Flecha generada en placa tetrapoyada por una carga uniforme de valor p w<sup>S</sup><sub>p</sub>(x,y)** (aquí denominada w_S_xy)

In [ ]:
## Flecha para caso de carga constante uniformemente distribuida p

#p = 1  # Haz p=1 si estás usando esta celda en una fase de cálculo de la densidad del material granular, en cualquier otro caso, deja esta
        # línea comentada para que el valor de p guardado en memoria al principio del script sea el utilizado para el cálculo de la flecha
    

# Solución de la EDO homogénea wn_S_h(x)
Bn = 2*p/(b*D*lambda_n**5*np.cosh(alpha_n))
Cn = -Bn*(2+alpha_n*np.tanh(alpha_n))
wn_S_h = Bn*lambda_n*X*np.sinh(lambda_n*X) + Cn*np.cosh(lambda_n*X)

# Solución de la EDO particular wn_S_p(x)
wn_S_p = 4*p/(b*D*lambda_n**5)

# Solución general de los términos w_S(x) de la serie de Levy
wn_S = wn_S_h + wn_S_p

# Flecha w_S_xy_P
w_S_xy = np.zeros((len(x), len(y)))
for j in range(len(y)):
    w_S_xy[:, j] = np.sum(wn_S*np.sin(lambda_n*y[j]), axis=1)

## Representemos la flecha calculada
imag_S = plt.imshow(w_S_xy*1e3, cmap='viridis', interpolation='nearest', extent=[y.min(), y.max(), x.max(), x.min()])
plt.suptitle('w(x,y)_S', x=0.57, y=0.95)
plt.ylabel('Dimension x') # Nota que los ejes están invertidos para acomodarse a la forma de representar la placa
plt.xlabel('Dimension y')
plt.colorbar(imag_S,label='Flecha (mm)')
plt.show()

## Evalua la funcion w(x,y)_S en un punto de coordenadas (x_eval,y_eval). Introduce el valor en x_val e y_val
x_eval = ...
y_eval = ...
x_eval_idx = np.argmin(np.abs(x-x_eval))
y_eval_idx = np.argmin(np.abs(y-y_eval))
w_S_xy_eval = w_S_xy[x_eval_idx,y_eval_idx]
print('El valor de la flecha en el punto evaluado', '(', x_eval, ',', y_eval, ')', 'es:', "{:e}".format(w_S_xy_eval), 'm')

<br>

**4. Flecha generada en placa tetrapoyada por una carga lineal de valor máximo q, w<sup>A</sup><sub>q</sub>(x,y)** 
(aquí denominada w_A_xy)

In [ ]:
## Flecha para caso de carga lineal de valor máximo q

#q = 1  # Haz q=1 si estás usando esta celda en una fase de cálculo de la densidad del material granular, en cualquier otro caso, deja esta
        # línea comentada para que el valor de p guardado en memoria al principio del script sea el utilizado para el cálculo de la flecha

# Solución de la EDO homogénea wn_A_h(x)
Dn = 2*q/(b*D*lambda_n**5*np.sinh(alpha_n))
An = -Dn*(2+alpha_n*(1/np.tanh(alpha_n)))
wn_A_h = An*np.sinh(lambda_n*X) + Dn*lambda_n*X*np.cosh(lambda_n*X)

# Solución de la EDO particular wn_A_p(x)
wn_A_p = 8*q*X/(a*b*D*lambda_n**5)

# Solución general de los términos w_A(x) de la serie de Levy
wn_A = wn_A_h + wn_A_p

# Flecha w_A_xy_q
w_A_xy = np.zeros((len(x), len(y)))
for j in range(len(y)):
    w_A_xy[:, j] = np.sum(wn_A*np.sin(lambda_n*y[j]), axis=1)

## Representemos la flecha calculada
imag_A = plt.imshow(w_A_xy*1e3, cmap='viridis', interpolation='nearest', extent=[y.min(), y.max(), x.max(), x.min()])
plt.suptitle('w(x,y)_A', x=0.57, y=0.95)
plt.ylabel('Dimension x') # Nota que los ejes están invertidos para acomodarse a la forma de representar la placa
plt.xlabel('Dimension y')
plt.colorbar(imag_A,label='Flecha (mm)')
plt.show()

## Evalua la funcion w(x,y)_A en un punto de coordenadas (x_eval,y_eval). Introduce el valor en x_val e y_val
x_eval = ...
y_eval = ...
x_eval_idx = np.argmin(np.abs(x-x_eval))
y_eval_idx = np.argmin(np.abs(y-y_eval))
w_A_xy_eval = w_A_xy[x_eval_idx,y_eval_idx]
print('El valor de la flecha en el punto evaluado', '(', x_eval, ',', y_eval, ')', 'es:', "{:e}".format(w_A_xy_eval), 'm')

## Encuentra el valor de la flecha máxima en valores absolutos
w_max = np.abs(w_A_xy).max()
print('El valor de la flecha máxima en valor absoluto es:', "{:e}".format(w_max), 'm')
w_max_idx = np.abs(w_A_xy).argmax()
w_max_x_idx,w_max_y_idx = np.unravel_index(w_max_idx,w_A_xy.shape)
w_max_x = w_max_x_idx*dx
w_max_y = w_max_y_idx*dy
print('Las coordenadas (x,y) donde se da el máximo son:', '(', w_max_x, ',', w_max_y, ') m')

<br>

**5. Flecha final como suma de los estados simétrico y antimétrico**

In [ ]:
# Flecha total w(xy) como suma de las flechas de los estados simétrico y antimétrico
w_xy = w_S_xy + w_A_xy

## Representemos la flecha calculada
imag = plt.imshow(w_xy, cmap='viridis', interpolation='nearest', extent=[y.min(), y.max(), x.max(), x.min()])
plt.suptitle('w(x,y)', x=0.57, y=0.95)
plt.ylabel('Dimension x') # Nota que los ejes están invertidos para acomodarse a la forma de representar la placa
plt.xlabel('Dimension y')
plt.colorbar(imag,label='Flecha (mm)')
plt.show()

## Evalua la funcion w(x,y)_A_q en un punto de coordenadas (x_eval,y_eval). Introduce el valor en x_val e y_val
x_eval = ...
y_eval = ...
x_eval_idx = np.argmin(np.abs(x-x_eval))
y_eval_idx = np.argmin(np.abs(y-y_eval))
w_xy_eval = w_xy[x_eval_idx,y_eval_idx]
print('El valor de la flecha en el punto evaluado', '(', x_eval, ',', y_eval, ')', 'es:', "{:e}".format(w_xy_eval), 'm')